In [1]:
# use new database file to plot
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from shapely import wkb
from binascii import unhexlify
from codecs import encode
import pysal

import seaborn as sns
%matplotlib tk

In [2]:
db_name = "data/USA_osm.db"

conn = sqlite3.connect(db_name)
conn.enable_load_extension(True)
conn.load_extension("mod_spatialite")

In [19]:
query = """
        SELECT db.ROWID, db.tunnel_dense as tunnel_dense, db.num_tunnel as num_tunnel, db.county_name as county_name, Hex(ST_AsBinary(db.geometry)) as geometry
        FROM county_tunnel_info as db
        """
tunnel_map = gpd.GeoDataFrame.from_postgis(query, conn, geom_col='geometry',crs={"init":"epsg:4326"})
tunnel_map.head(5)

,rowid,tunnel_dense,num_tunnel,county_name,geometry
0,1,0.000000,0,Daniels,"(POLYGON ((-106.15193 48.910637, -106.112003 4..."
1,2,0.000000,0,Bristol,"(POLYGON ((-71.353897 41.751301, -71.347183 41..."
2,3,0.021565,2,Brooke,"(POLYGON ((-80.67960000000001 40.186371, -80.6..."
3,4,0.000000,0,Nassau,"(POLYGON ((-82.050432 30.676266, -82.041811999..."
4,5,0.001035,1,Robeson,"(POLYGON ((-79.4617538654129 34.6304321484729,..."


In [3]:
# plot the whole tunnel distribution in USA
# make the colormap
from branca.colormap import linear,LinearColormap
import math

n=len(tunnel_map)
colormap = linear.YlOrRd.scale(tunnel_map['num_tunnel'].min(), tunnel_map['num_tunnel'].max())
value_list = range(tunnel_map['num_tunnel'].min()+1, tunnel_map['num_tunnel'].max()+1)
# add a log10 index, and change the color map to stepcolormap
max_ = max(value_list)
min_ = min(value_list)
index = [5**(math.log10(min_) + i * (math.log10(max_) -math.log10(min_)) * 1./n) for i in range(1+n)]
log_tunnel_colormap = colormap.to_step(index=index, round_method='int')
log_tunnel_colormap

In [4]:
# convert the table to a dictionary to map a feature to it's number of tunnels
tunnel_map_dict = tunnel_map.set_index('rowid')['num_tunnel']

In [5]:
# plot the whole tunnel distribution in USA
# make the colormap
from branca.colormap import linear,LinearColormap
import math

n=len(tunnel_map)
colormap = linear.YlOrRd.scale(tunnel_map['tunnel_dense'].min(), tunnel_map['tunnel_dense'].max())
value_list = np.arange(tunnel_map['tunnel_dense'].min()+0.001, tunnel_map['tunnel_dense'].max()+0.001).tolist()
# add a log10 index, and change the color map to stepcolormap
max_ = max(value_list)
min_ = min(value_list)
index = [10**(math.log10(min_) + i * (math.log10(max_) -math.log10(min_)) * 1./n) for i in range(1+n)]
log_tunnel_dense_colormap = colormap.to_step(index=index, round_method='int')
log_tunnel_dense_colormap

In [6]:
# convert the table to a dictionary to map a feature to it's number of tunnels
tunnel_dense_map_dict = tunnel_map.set_index('rowid')['tunnel_dense']

In [7]:
m = folium.Map(location=[42.5, -83.5], zoom_start=7, tiles="cartodbpositron")
folium.GeoJson(tunnel_map,
               name='tunnel_num',
               style_function=lambda feature: {
                   'fillColor': log_tunnel_colormap(tunnel_map_dict[feature['properties']['rowid']]+1),
                   #'fillColor': colors.to_hex(colormap.to_rgba(county_map_dict[feature['properties']['rowid']]+1)),
                   #'color': 'black',
                   'weight': 1,
                   'dashArray': '5, 5',
                   'fillOpacity': 0.9,
                }
              ).add_to(m)
folium.GeoJson(tunnel_map,
               name='tunnel_dense',
               style_function=lambda feature: {
                   'fillColor': log_tunnel_dense_colormap(tunnel_dense_map_dict[feature['properties']['rowid']]+1),
                   #'fillColor': colors.to_hex(colormap.to_rgba(county_map_dict[feature['properties']['rowid']]+1)),
                   #'color': 'black',
                   'weight': 1,
                   'dashArray': '5, 5',
                   'fillOpacity': 0.9,
                }
              ).add_to(m)


# add legend
log_tunnel_colormap.caption = '# of Tunnels'
log_tunnel_colormap.add_to(m)
log_tunnel_dense_colormap.caption = 'Density of Tunnels (/sqmile)'
log_tunnel_dense_colormap.add_to(m)

#m.save('dense2.html')


In [10]:
# ---------------------------------------- plot accident layer ----------------------------------------------
query = """
        SELECT ROWID as rowid, num_accident , Hex(ST_AsBinary(geometry)) as geometry
        FROM county_accident
        """
accident_map = gpd.GeoDataFrame.from_postgis(query, conn, geom_col='geometry',crs={"init":"epsg:4326"})
accident_map.head(5)
#folium.Circle(location=(-83,42)).add_to(m)

In [9]:
# plot the whole tunnel distribution in USA
# make the colormap
from branca.colormap import linear,LinearColormap
import math

n=len(accident_map)
colormap = linear.YlGnBu.scale(accident_map['num_accident'].min(), accident_map['num_accident'].max())
value_list = range(accident_map['num_accident'].min()+1, accident_map['num_accident'].max()+1)
# add a log10 index, and change the color map to stepcolormap
max_ = max(value_list)
min_ = min(value_list)
index = [5**(math.log10(min_) + i * (math.log10(max_) -math.log10(min_)) * 1./n) for i in range(1+n)]
log_accident_colormap = colormap.to_step(index=index, round_method='int')
log_accident_colormap

In [10]:
# convert the table to a dictionary to map a feature to it's number of tunnels
county_accident_map_dict = accident_map.set_index('rowid')['num_accident']

In [11]:
folium.GeoJson(accident_map,
               name='num_accident',
               style_function=lambda feature: {
                   'fillColor': log_accident_colormap(county_accident_map_dict[feature['properties']['rowid']]+1),
                   #'fillColor': colors.to_hex(colormap.to_rgba(county_map_dict[feature['properties']['rowid']]+1)),
                   #'color': 'black',
                   'weight': 1,
                   'dashArray': '5, 5',
                   'fillOpacity': 0.9,
                }
              ).add_to(m)


# add legend
log_accident_colormap.caption = '# of Accidents'
log_accident_colormap.add_to(m)
folium.LayerControl().add_to(m)

m.save('tunnel_accident.html')

In [3]:
#----------------------------------------traffic statistics ----------------------------------------------
query = """
        SELECT ac.HOUR, ac.MONTH, ac.COUNTY, ac.DAY_WEEK, ac.ROAD_FNC, ac.ROUTE
        FROM Fatal_Motor_Vehicle_Accidents as ac
        
        """
res = conn.execute(query)
data = res.fetchall()
#print (month_distribution)

In [21]:
data_column = {}
data_column['month'] = []
data_column['day_week'] = []
data_column['hour'] = []
data_column['county'] = []
data_column['road_fnc'] = []
data_column['road_fnc'] = []
for i, frame in enumerate (data):
    data_column['road_fnc'].append(frame[4])
    data_column['day_week'].append(frame[3])
    data_column['county'].append(frame[2])
    data_column['month'].append(frame[1])
    data_column['hour'].append(frame[0])
    

In [23]:
sns.set(style="darkgrid")
f = plt.figure(figsize=(16,8.4),dpi=80)
ax = f.add_subplot(111)
ax = sns.countplot(x='road_fnc', data = data_column, ax=ax)
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 1.1, box.height])
#         ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)

# add numbers
for p in ax.patches:
    ax.text(p.get_width() + 10000,
            p.get_y()+p.get_height()*0.3,
            '{:1.5f}'.format(p.get_width()),
            ha="center", rotation=-90)
#ax.set_yticklabels(ax.get_yticklabels(), fontsize=16)
plt.xlabel('Road functional classes')
#plt.xlabel('Month in a year')
plt.ylabel('Number of accidents')
plt.savefig('output/road_fnc_distribution.png')

In [33]:
# ---------------------------------------- plot traffic count layer ----------------------------------------------
query = """
        SELECT PK_UID as rowid, AADT11, AADT12, Hex(ST_AsBinary(ST_Transform(geometry,4326))) as geometry
        FROM NTFA_Segment_3
        """
traffic_count_map = gpd.GeoDataFrame.from_postgis(query, conn, geom_col='geometry',crs={"init":"epsg:4326"})
traffic_count_map.head(5)

,rowid,AADT11,AADT12,geometry
0,1,184,227,LINESTRING (-83.03116108950763 43.081582988294...
1,2,941,941,LINESTRING (-82.53101248758864 43.268360248761...
2,3,15384,14100,LINESTRING (-83.40906226497388 41.920036616977...
3,4,21415,21415,LINESTRING (-85.19860043482129 42.261039863256...
4,5,5323,6069,LINESTRING (-83.69345485313606 43.020251600947...


In [43]:
from branca.colormap import linear,LinearColormap
import math

n=len(traffic_count_map)
colormap = linear.YlOrRd.scale(traffic_count_map['AADT12'].min(), traffic_count_map['AADT12'].max())
value_list = np.arange(traffic_count_map['AADT12'].min()+1, traffic_count_map['AADT12'].max()+1).tolist()
# add a log10 index, and change the color map to stepcolormap
max_ = max(value_list)
min_ = min(value_list)
index = [5**(math.log10(min_) + i * (math.log10(max_) -math.log10(min_)) * 1./n) for i in range(1+n)]
log_AADT_colormap = colormap.to_step(index=index, round_method='int')
log_AADT_colormap

In [44]:
AADT_map_dict = traffic_count_map.set_index('rowid')['AADT12']

In [45]:
m = folium.Map(location=[42.5, -83.5], zoom_start=7, tiles="cartodbpositron")
folium.GeoJson(traffic_count_map,
               name='AADT map',
               style_function=lambda feature: {
                   'fillColor': log_AADT_colormap(AADT_map_dict[feature['properties']['rowid']]),
                   #'fillColor': colors.to_hex(colormap.to_rgba(county_map_dict[feature['properties']['rowid']]+1)),
                   #'color': 'black',
                   'weight': 1,
                   'dashArray': '5, 5',
                   'fillOpacity': 0.9,
                }
              ).add_to(m)

In [46]:
# add legend
log_AADT_colormap.caption = 'Annual average daily traffic 2012'
log_AADT_colormap.add_to(m)
folium.LayerControl().add_to(m)

m.save('AADT_michigan_2012.html')